In [1]:
#import streamlit as st
#import pandas as pd
#import altair as alt
#import plotly.express as px

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from datetime import datetime
import matplotlib.dates as mdates
import matplotlib.patches as mpatches
import matplotlib.lines as mlines
import matplotlib.ticker as mtick
import os
import platform
#import mercury as mr

In [3]:
if platform.system()=='Windows': 
    #Directorios Windows
    os.chdir(
        r"G:\.shortcut-targets-by-id\120bUa35y3FJzvumWahpOx1qS9I6TIrhb\GIGANAV")

if platform.system()=='Darwin':
    os.chdir(
        '/Users/juliandiazparra/Library/CloudStorage/GoogleDrive-julian.adiaz@hotmail.com/Mi unidad/Negocios/Familiar/GIGANAV')

folder=os.getcwd()
data=os.path.join(folder,'Datos')
provisiones=os.path.join(data,'Provisiones')
facturas=os.path.join(data,'Facturas')
tickets=os.path.join(data,'Tickets')
usuarios=os.path.join(data,'Usuarios')
egresos=os.path.join(data,'Egresos')
transacciones=os.path.join(data,'Transacciones')
analisis=os.path.join(folder,'Operacion/Analisis')
resultados=os.path.join(analisis,'Resultados')
bases=os.path.join(analisis,'Bases_analisis')
outputs=os.path.join(analisis,'Outputs')
plots=os.path.join(outputs,'Gráficos')
usuarios_clean=os.path.join(bases,'Usuarios')
transac_clean=os.path.join(bases,'Transacciones')
tickets_clean=os.path.join(bases,'Tickets')
facturas_clean=os.path.join(bases,'Facturas')
egresos_clean=os.path.join(bases,'Egresos')
working=os.path.join(bases,'Bases_trabajo')
codigo=os.path.join(folder,'Operacion/Analisis/Código')
conciliaciones=os.path.join(folder,'Operacion/Operativo/Administrativo/Conciliaciones')

## Importación datos

## Tickets

In [4]:
#Tickets
years=[2022, 2023, 2024]
tickets_dict={}
tickets_append=pd.DataFrame()

for i in years:
    file_path=os.path.join(tickets_clean, 'Tickets_'+str(i))
    tickets_dict[i]=pd.read_csv(file_path+'.csv')
    tickets_append=pd.concat([
        tickets_dict[i],tickets_append],
                             ignore_index=True)
    
tickets_append.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14601 entries, 0 to 14600
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id_ticket      14601 non-null  int64 
 1   departamento   14601 non-null  object
 2   nombre         14601 non-null  object
 3   asunto         14601 non-null  object
 4   tecnico        14601 non-null  object
 5   zona           14564 non-null  object
 6   abierto_por    14601 non-null  object
 7   instal         14601 non-null  int64 
 8   soporte        14601 non-null  int64 
 9   traslado       14601 non-null  int64 
 10  retiro         14601 non-null  int64 
 11  mantenimiento  14601 non-null  int64 
 12  facturacion    14601 non-null  int64 
 13  cambio_plan    14601 non-null  int64 
 14  suspendido     14601 non-null  int64 
 15  tv             14601 non-null  int64 
 16  llamada        14601 non-null  int64 
 17  categoria      13792 non-null  object
 18  fecha_abierto  14601 non-n

In [5]:
for i in ['fecha_abierto', 'fecha_cerrado']:
    tickets_append.loc[
        :,i]=pd.to_datetime(tickets_append[i], 
                            format='%Y-%m-%d %H:%M:%S')

#Creo mes y semana de cerrado
tickets_append.loc[
    :,'mes_crd']=pd.to_datetime(tickets_append[
    'fecha_cerrado']).dt.strftime('%Y-%m')

tickets_append.loc[
    :,'first_day']=tickets_append[
    'fecha_cerrado'] - pd.to_datetime(tickets_append[
    'fecha_cerrado']).dt.weekday * np.timedelta64(
    1, 'D')

tickets_append.loc[
    :,'first_day']=pd.to_datetime(tickets_append[
    'first_day']).dt.strftime(
    '%Y-%m-%d')

#Tiempo de resolucion
tickets_append.loc[
    :,'tiempo_resolucion']=tickets_append[
    'fecha_cerrado']-tickets_append[
    'fecha_abierto']

#Creo departamentos
tickets_append.loc[
    :, 'cat_tecnico']=tickets_append[
    'categoria'].str.contains(
    'Instalacion|Retiro|Soporte|Traslado|Mantenimiento', 
    case=False)*1

tickets_append.loc[
    (tickets_append['cat_tecnico']==1), 
    'departamento']='Tecnico'

tickets_append.loc[
    (tickets_append['categoria'].isna()), 
    'departamento']='Otro'

tickets_append.loc[
    :, 'comercial']=tickets_append[
    'categoria'].str.contains(
    'Facturacion|Cambio de plan|Suspendido', 
    case=False)*1
tickets_append.loc[
    tickets_append['comercial']==1, 
    'departamento']='Comercial'

/var/folders/_1/wccngqsd417dbsr4pcv82tj80000gn/T/ipykernel_8202/419886152.py:12: PerformanceWarning: Adding/subtracting object-dtype array to TimedeltaArray not vectorized.
  :,'first_day']=tickets_append[


## Usuarios

In [6]:
#Importo la base de usuarios más reciente que tenga
import glob
import os
list_of_files = glob.glob(os.path.join(usuarios_clean,'*.csv'))
latest_file = max(list_of_files, key=os.path.getctime)
base_usuarios=pd.read_csv(latest_file)

In [7]:
#Variables identificadoras usuarios
base_usuarios.loc[
    :,'tecnologia'
]=np.select([(
    base_usuarios['radio']==1)],['radio'], 
            default='fibra')

base_usuarios.loc[
    :,'tipo_servicio'
]=np.select([(
    base_usuarios['Tipo estrato']!=4)],['residencial'], 
            default='corporativo')

base_usuarios.loc[
    :,'tipo_usuario'
]=np.select([(
    base_usuarios['TV']==1)],['paquete'], 
            default='internet')

In [8]:
base_usuarios.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3764 entries, 0 to 3763
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Id                3764 non-null   int64  
 1   nombre            3764 non-null   object 
 2   estado            3764 non-null   object 
 3   radio             3764 non-null   int64  
 4   Tipo estrato      3764 non-null   int64  
 5   megas             3764 non-null   int64  
 6   TV                3764 non-null   int64  
 7   Telefono          2803 non-null   object 
 8   Movil             3699 non-null   object 
 9   Correo            2839 non-null   object 
 10  Zona              3740 non-null   object 
 11  tarifa            3764 non-null   float64
 12  Deuda actual      1129 non-null   object 
 13  ID de registro    3757 non-null   float64
 14  meses_suspendido  190 non-null    float64
 15  deuda             190 non-null    float64
 16  tecnologia        3764 non-null   object 


## Material

In [11]:
spreadsheet_id = "1OoFik80Zcs6ih83j4_RrSdZAKaZNp9XyV-iMWiI1Vo4"
sheet_id = "26881671"
url = f"https://docs.google.com/spreadsheets/d/{spreadsheet_id}/export?gid={sheet_id}&format=csv"
uso_material = pd.read_csv(url)
uso_material.head()

URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1000)>

In [ ]:
#Limpio datos
#ID tickets
uso_material=uso_material.loc[
    uso_material['Número de ticket'].notna()]

uso_material.loc[:,'id_ticket'] =uso_material[
    'Número de ticket'].str.replace(
    '#','').str.lstrip('0').astype(int)

#Separo tecnicos responsables
uso_material[[
    'tecnico_1', 'tecnico_2'
]]=uso_material[
    'Técnicos involucrados'].str.split(
    ',', expand=True).fillna('')

#Creo variable de mes de la fecha
uso_material.loc[:,'timestamp']=pd.to_datetime(uso_material['Marca temporal'], 
                                         format='%d/%m/%Y %H:%M:%S')
uso_material.loc[:,'month_pago']=uso_material['timestamp'].dt.strftime('%Y-%m')

base_material=uso_material.iloc[:,np.r_[20,21:23,3:8,14:16,8:13,19:20]]
base_material=base_material.fillna(0)
base_material.head()

In [ ]:
base_material.info()

In [ ]:
materiales_names=['conector_mecanico', 'fibra_drop', 'conector_coaxial', 
                  'tensores', 'cable_coaxial', 'cable_utp','rjs',
                  'deco_tdt', 'splitter_2', 'splitter_3', 'splitter_4', 
                  'mininodos', 'otros']

for x,y in enumerate(list(base_material.iloc[:,3:16].columns)):
    #Lleno vacios y elimino todos los caracteres no numericos
    base_material.loc[:,y]=base_material[y].astype(str).str.extract(r'(\d+)', expand=False).fillna(0).astype(int)
    base_material.rename(columns={y:materiales_names[x]}, inplace=True)

base_material.info()

## Encuesta satisfacción

In [ ]:
spreadsheet_id = "1f5YsOXwh52gKB_qyll0D5GR8AbYrGzhJSgIAB_PrC4s"
sheet_id = "1721707425"
url = f"https://docs.google.com/spreadsheets/d/{spreadsheet_id}/export?gid={sheet_id}&format=csv"
enc_satisfaccion = pd.read_csv(url)
enc_satisfaccion.info()

In [ ]:
#Depuro variables y acomodo id ticket
enc_satisfaccion.loc[
    :,'id_ticket']=enc_satisfaccion.iloc[:,7].combine_first(
        enc_satisfaccion.iloc[:,10])

base_satisfaccion=enc_satisfaccion.loc[
    enc_satisfaccion['id_ticket'].notna()].iloc[
    :,[0,-1,4,5,6,9]]

base_satisfaccion.loc[
    :,'id_ticket']=base_satisfaccion[
    'id_ticket'].astype(float).astype(int)

base_satisfaccion.columns=['fecha_enc_sat','id_ticket',
                           'calificacion_personal','resolvio',
                           'promoter','calificacion_servicio']
base_satisfaccion.info()

In [ ]:
for v in list(base_satisfaccion.iloc[:,[3,4]].columns):
    base_satisfaccion.loc[
        :,v
    ]=(base_satisfaccion[v]=='Sí')*1

base_satisfaccion.head()

## Setup

In [ ]:
#Pego con base de usuarios
tickets_append[
    'nombre']=tickets_append[
    'nombre'].str.strip()

base_usuarios[
    'nombre']=base_usuarios[
    'nombre'].str.strip()

working_data=tickets_append.merge(
    base_usuarios, on='nombre', how='left', 
    indicator=True).merge(
    base_material, on='id_ticket', how='left', 
    indicator='merge2').merge(
    base_satisfaccion,on='id_ticket',how='left',
    indicator='merge_sat')

working_data.info()

In [ ]:
#Creo variables de ajuste
working_data.loc[:,'encuesta']=working_data['merge2']=='both'*1

#Creo variable de residencial
working_data.loc[:,'residencial']=(working_data['Tipo estrato']!=4)*1

#Calculo tiempo de respuesta, numero de tickets y respuestas a la encuesta
working_data['tiempo_resol']=working_data['tiempo_resolucion']/ pd.Timedelta(days=1)

In [ ]:
working_data.loc[working_data['cat_tecnico']==1].groupby(
    ['mes_crd','tipo_servicio','categoria']).agg(
    num_tickets=('id_ticket','nunique'), 
    tiempo_respuesta=('tiempo_resol',pd.Series.median)
).reset_index()

In [ ]:
#analisis residencia/empresa
summary_res_emp=working_data.loc[working_data['cat_tecnico']==1].groupby(
    ['mes_crd','tipo_servicio','categoria']).agg(
    num_tickets=('id_ticket','nunique'), 
    tiempo_respuesta=('tiempo_resol',pd.Series.mean)).reset_index().pivot(
    index='mes_crd',columns=[
        'categoria','tipo_servicio'], 
    values=['num_tickets',
           'tiempo_respuesta']).reset_index()
summary_res_emp.tail()

In [ ]:
#Analisis TV o internet
summary_int_tv=working_data.loc[working_data['cat_tecnico']==1].groupby(
    ['mes_crd','tipo_usuario','categoria']).agg(
    num_tickets=('id_ticket','nunique'), 
    tiempo_respuesta=('tiempo_resol',pd.Series.mean)).reset_index().pivot(
    index='mes_crd',columns=[
        'categoria','tipo_usuario'], 
    values=['num_tickets',
           'tiempo_respuesta']).reset_index()
summary_int_tv.tail(10)

In [ ]:
summary_fibra_radio=working_data.loc[
    working_data['cat_tecnico']==1].groupby(
    ['mes_crd', 'categoria', 'tecnologia']).agg(
    tiempo_respuesta=('tiempo_resol',pd.Series.mean), 
    num_tickets=('id_ticket', 'nunique'), 
    encuestas=('encuesta', 'sum')).reset_index().pivot(
    index='mes_crd', columns=[
        'categoria', 'tecnologia'], values=[
        'tiempo_respuesta', 'num_tickets', 'encuestas'
    ]).reset_index()

summary_fibra_radio.tail()

In [ ]:
summary_encuesta_cat=working_data.loc[
    working_data['cat_tecnico']==1
].groupby([
    'mes_crd','categoria'
]).agg(num_tickets=(
    'id_ticket','nunique'), 
       encuesta=(
           'encuesta','sum')).reset_index().pivot(
    columns='categoria',
    index='mes_crd',values=[
        'num_tickets',
        'encuesta']).reset_index()

for i in ['Instalacion','Retiro','Soporte','Traslado']:
    summary_encuesta_cat.loc[
        :,'per_enc_'+i]=summary_encuesta_cat[
        ('encuesta',i)]/summary_encuesta_cat[
        ('num_tickets',i)]
summary_encuesta_cat=summary_encuesta_cat.iloc[:,np.r_[0,-4:0]]

nombre_columnas=[]
for i, x in enumerate(list(summary_encuesta_cat.columns)):
    nombre_columnas.append(x[0])

summary_encuesta_cat.columns=nombre_columnas
summary_encuesta_cat.tail()

In [ ]:
for i in [summary_int_tv,summary_fibra_radio,summary_res_emp,]:
    nombres_columnas=[]
    for k, x  in enumerate(list(i.columns)):            
        nombres_columnas.append('_'.join([str(j) for j in list(x)]))
    i.columns=nombres_columnas
    i.rename(columns={'mes_crd__':'mes_crd'}, inplace=True)

In [ ]:
working_summary=summary_fibra_radio.merge(
    summary_int_tv, on='mes_crd',how='inner').merge(
    summary_res_emp, on='mes_crd',how='inner').merge(
    summary_encuesta_cat, on='mes_crd',how='inner')
working_summary.tail()

## Analisis fibra vs radio

In [ ]:
fig, ax= plt.subplots()

working_summary.iloc[-5:].plot(x='mes_crd', y='tiempo_respuesta_Soporte_fibra', 
                               kind='line', ax=ax, 
                               color='magenta', grid=True)
working_summary.iloc[-5:].plot(x='mes_crd', y='tiempo_respuesta_Instalacion_fibra', 
                             kind='line', ax=ax, 
                               color='blue', grid=True)
working_summary.iloc[-5:].plot(x='mes_crd', y='tiempo_respuesta_Traslado_fibra', 
                             kind='line', ax=ax, 
                               color='grey', grid=True)
working_summary.iloc[-5:].plot(x='mes_crd', y='tiempo_respuesta_Retiro_fibra', 
                             kind='line', ax=ax, 
                               color='purple', grid=True)


plt.setp(ax, ylabel='Días', xlabel='', title='Promedio de tiempo de respuesta: fibra')
ax.legend(['Soporte', 'Instalacion', 'Traslado','Retiro'])
plt.tight_layout()
plt.show()

In [ ]:
summary_tix_fibra_radio=working_data.loc[
    working_data['cat_tecnico']==1].groupby(
    ['mes_crd', 'tecnologia']).agg(
    num_tix=('id_ticket','nunique')).reset_index().pivot(
    columns='tecnologia',
    values='num_tix', 
    index='mes_crd').reset_index()
summary_tix_fibra_radio.columns=['mes_crd','fibra','radio']
summary_tix_fibra_radio.loc[
    :,'porc_fibra'
]=summary_tix_fibra_radio[
    'fibra']/(summary_tix_fibra_radio[
    'fibra']+summary_tix_fibra_radio[
    'radio'])

summary_tix_fibra_radio.loc[
    :,'porc_radio'
]=summary_tix_fibra_radio[
    'radio']/(summary_tix_fibra_radio[
    'fibra']+summary_tix_fibra_radio[
    'radio'])
summary_tix_fibra_radio.tail()
#print('Durante el último trimestre, el '+str(
#    summary_tix_fibra_radio.iloc[-4:-1][
#        'porc_fibra'].mean())+' de tickets fueron de fibra')

In [ ]:
fig,ax=plt.subplots()
summary_tix_fibra_radio.iloc[-6:-1,:3].plot.bar(
    'mes_crd','fibra',
    ax=ax, #bottom=base, 
    color='lightseagreen', 
    grid=True)
summary_tix_fibra_radio.iloc[-6:-1,:3].plot.bar(
    'mes_crd','radio',
    ax=ax, bottom=summary_tix_fibra_radio.iloc[-6:-1,:3]['fibra'], 
    color='tomato', 
    grid=True)

plt.tight_layout()
plt.show()

In [ ]:
fig, ax=plt.subplots()

working_summary.iloc[-6:].plot(
    x='mes_crd', y='tiempo_respuesta_Soporte_radio', 
    kind='line', ax=ax, 
    color='magenta', grid=True, linestyle='--')

working_summary.iloc[-6:].plot(
    x='mes_crd', y='tiempo_respuesta_Instalacion_radio', 
                               kind='line', ax=ax, 
                               color='blue', grid=True, linestyle='--')

working_summary.iloc[-6:].plot(
    x='mes_crd', y='tiempo_respuesta_Traslado_radio', 
                               kind='line', ax=ax, 
                               color='grey', grid=True, linestyle='--')

working_summary.iloc[-6:].plot(
    x='mes_crd', y='tiempo_respuesta_Retiro_radio', 
                               kind='line', ax=ax, 
                               color='purple', grid=True, linestyle='--')

plt.setp(ax, ylabel='Días', xlabel='', title='Promedio de tiempo de respuesta: radio')
ax.legend(['Soporte', 'Instalacion', 'Traslado','Retiro'])
plt.tight_layout()
plt.show()

## Resumen Indicadores Coordinador

In [ ]:
#Hago resumen Coordinador
working_summary.loc[
    :,'per_encuestas'
]=working_summary.loc[
    :,['per_enc_Instalacion',
       'per_enc_Soporte']].mean(axis=1)

In [ ]:
#Indicadores
valor_variable=380000
indicadores_tecnicos=pd.DataFrame()
indicadores_tecnicos.loc[
    :,'Indicadores']=['tiempo_respuesta_Soporte_fibra',
        'tiempo_respuesta_Instalacion_fibra', 
        'per_encuestas']
indicadores_tecnicos.loc[
    :,'Peso'
]=[.4,.4,.2]
indicadores_tecnicos.loc[
    :,'Valor'
]=indicadores_tecnicos[
    'Peso']*valor_variable

indicadores_tecnicos

In [ ]:
mes=list(working_summary.loc[pd.to_datetime(
    working_summary['mes_crd']).dt.strftime(
    '%Y')==datetime.today().strftime(
    '%Y'), 'mes_crd'].unique())

filepath=os.path.join(
    outputs,
    'Indicadores_coordinador_'+datetime.today().strftime('%Y')+'.xlsx')

writer = pd.ExcelWriter(filepath, 
                        engine = 'xlsxwriter') 

for m in mes:
    resumen_coordinador=pd.DataFrame()
    resumen_coordinador.loc[
        :,m
    ]=working_summary.loc[
        working_summary['mes_crd']==m, [
            'mes_crd', 
            'tiempo_respuesta_Soporte_fibra',
            'tiempo_respuesta_Instalacion_fibra', 
            'per_encuestas'
        ]
    ].iloc[-1].iloc[1:]

    resumen_coordinador.loc[
        :,'Meta'
    ]=[3.5,3.5,.7]

    resumen_coordinador.loc[
        :,'% cumplimiento'
    ]=1-((resumen_coordinador[
        m]-resumen_coordinador[
        'Meta'])/resumen_coordinador[
        'Meta'])

    resumen_coordinador.loc[
        resumen_coordinador.index=='per_encuestas', 
        '% cumplimiento']=resumen_coordinador[
        m]/resumen_coordinador['Meta']

    resumen_coordinador.reset_index(inplace=True)
    resumen_coordinador.rename(
        columns={'index':'Indicadores'}, inplace=True)
    resumen_coordinador=resumen_coordinador.merge(
        indicadores_tecnicos[['Indicadores','Valor']], 
        on='Indicadores')

    #Creo compensaciones
    resumen_coordinador.loc[
        :,'Compensacion']=np.select(
        [(resumen_coordinador['% cumplimiento']<.8)] ,[0],
        default=(resumen_coordinador[
            '% cumplimiento']*resumen_coordinador[
            'Valor']))
    
    resumen_coordinador.to_excel(
                writer, sheet_name=m, index=False)
    
writer.close()

In [ ]:
working_summary[['mes_crd','tiempo_respuesta_Instalacion_fibra',
                 'tiempo_respuesta_Instalacion_fibra','per_encuestas']].tail()

## Panel técnicos instaladores

In [ ]:
#Identifico ticket si cada tecnico aparece
working_data[['tecnico_1','tecnico_2']].fillna('', inplace=True)

#Creo variable de participacion de cada tecnico
for t in list(working_data.loc[
    working_data['tecnico_1'].notna(),
    'tecnico_1'].unique()):
    
    working_data.loc[
        :,t.split()[0]
    ]=np.select([(working_data['tecnico_1']==t) |
        (working_data['tecnico_2']==t)],[1],default=0)

In [ ]:
#Numero de tickets atendidos por tecnico/mes
num_tix_tecnicos=working_data.groupby(
    'mes_crd').agg(
    num_enc_Esneider=('Esneider','sum'), 
    num_enc_Alfonso=('Alfonso','sum'), 
    num_enc_Sair=('Sair','sum'), 
    num_enc_Leonardo=('Leonardo','sum'),
    num_enc_Jaime=('Jaime','sum')).reset_index().tail(8)#.iloc[:-1]
num_tix_tecnicos

In [ ]:
#Promedio tiempo de respuesta por tecnico/mes
temp_res_tecnicos_summary=working_data.loc[
    working_data['Esneider']==1
].groupby('mes_crd').agg(
    temp_res_esneider=('tiempo_resol',pd.Series.mean)).merge(
    working_data.loc[
    working_data['Alfonso']==1
].groupby('mes_crd').agg(
    temp_res_alfonso=('tiempo_resol',pd.Series.mean)), 
    left_index=True, right_index=True).merge(
    working_data.loc[
    working_data['Sair']==1
].groupby('mes_crd').agg(
    temp_res_sair=('tiempo_resol',pd.Series.mean)), 
    left_index=True, right_index=True).merge(
    working_data.loc[
    working_data['Leonardo']==1
].groupby('mes_crd').agg(
    temp_res_leonardo=('tiempo_resol',pd.Series.mean)), 
    left_index=True, right_index=True).reset_index()
temp_res_tecnicos_summary

In [ ]:
#Grafico tickets y tiempo respuesta promedio
fig,ax=plt.subplots()

num_tix_tecnicos.iloc[-4:].plot(
    x='mes_crd', y='num_enc_Esneider',
    kind='line',ax=ax,color='blue')

num_tix_tecnicos.iloc[-4:].plot(
    x='mes_crd', y='num_enc_Alfonso',
    kind='line',ax=ax,color='red')

num_tix_tecnicos.iloc[-4:].plot(
    x='mes_crd', y='num_enc_Sair',
    kind='line',ax=ax,color='black')

num_tix_tecnicos.iloc[-4:].plot(
    x='mes_crd', y='num_enc_Leonardo',
    kind='line',ax=ax,color='cyan')

ax.legend(['Esneider', 'Alfonso', 'Sair','Leonardo'])

plt.setp(ax, ylabel='Tickets', xlabel='')
plt.suptitle('Total tickets por técnico/mes\nGiganav Connections')
plt.tight_layout()
plt.show()

In [ ]:
#Grafico tickets y tiempo respuesta promedio
fig,ax=plt.subplots()

num_tix_tecnicos.iloc[-4:].plot(
    x='mes_crd', y='num_enc_Esneider',
    kind='line',ax=ax,color='blue')

num_tix_tecnicos.iloc[-4:].plot(
    x='mes_crd', y='num_enc_Alfonso',
    kind='line',ax=ax,color='red')

num_tix_tecnicos.iloc[-4:].plot(
    x='mes_crd', y='num_enc_Sair',
    kind='line',ax=ax,color='black')

num_tix_tecnicos.iloc[-4:].plot(
    x='mes_crd', y='num_enc_Leonardo',
    kind='line',ax=ax,color='cyan')

ax.legend(['Esneider', 'Alfonso', 'Sair','Leonardo'])

plt.setp(ax, ylabel='Tickets', xlabel='')
plt.suptitle('Total tickets por técnico/mes\nGiganav Connections')
plt.tight_layout()
plt.show()

In [ ]:
fig, ax=plt.subplots()

temp_res_tecnicos_summary.iloc[-4:].plot(
    x='mes_crd', y='temp_res_esneider',
    kind='line',ax=ax,color='blue', grid=True)

temp_res_tecnicos_summary.iloc[-4:].plot(
    x='mes_crd', y='temp_res_alfonso',
    kind='line',ax=ax,color='red', grid=True)

temp_res_tecnicos_summary.iloc[-4:].plot(
    x='mes_crd', y='temp_res_sair',
    kind='line',ax=ax,color='black')

temp_res_tecnicos_summary.iloc[-4:].plot(
    x='mes_crd', y='temp_res_leonardo',
    kind='line',ax=ax,color='cyan')

ax.legend(['Esneider', 'Alfonso', 'Sair','Leonardo'])

plt.setp(ax, ylabel='Días', xlabel='')
plt.suptitle('Tiempo de resolución por técnico/mes\nGiganav Connections')
plt.tight_layout()
plt.show()

Comparo con asignación de técnicos en Mikrowisp vs encuesta de material

In [ ]:
#Miro cuantos tickets de los asignados en Mikrowisp
mkwp_summary_month=working_data.groupby(['mes_crd','tecnico']).agg(
    tickets_asignados=('id_ticket','nunique'), 
    temp_res=('tiempo_resolucion','mean'),
    num_enc_satisfaccion=('fecha_enc_sat','count'),
    promedio_satisfaccion=('calificacion_personal','mean'),
    Promotores=('promoter','sum')).reset_index().fillna(0)

empleados_names=[
    'ESNEIDER GARCIA BARAJAS','ALFONSO ARANGO TORO','SAIR BOLAÑO DURAN',
    'LEONARDO QUEZADA RAMIREZ','JAIME LEONARDO BARRERA BARAJAS',
    'MAYRA FERNANDA TRUJILLO SUAREZ','KAROLINA GUERRERO PRINCE',
    'EDGARD FONSECA','MIGUELPACHECO','No asignado']

mkwp_summary_month=mkwp_summary_month.loc[mkwp_summary_month['tecnico'].isin(empleados_names)]
mkwp_summary_month.loc[
    mkwp_summary_month['mes_crd']=='2024-08'].sort_values(
        'tickets_asignados',ascending=False)

In [ ]:
enc_tecnicos_mes_long=pd.melt(num_tix_tecnicos,id_vars='mes_crd')

enc_tecnicos_mes_long.loc[
    :,'variable']=enc_tecnicos_mes_long['variable'].str.lstrip('num_enc_')

enc_tecnicos_mes_long.rename(
    columns={'variable':'tecnico', 'value':'Encuestas respondidas'}, inplace=True)

enc_tecnicos_mes_long.loc[
    :,'tecnico'
]=np.select([enc_tecnicos_mes_long['tecnico'].str.contains('esneider',case=False),
             enc_tecnicos_mes_long['tecnico'].str.contains('alfonso',case=False),
             enc_tecnicos_mes_long['tecnico'].str.contains('sair',case=False),
             enc_tecnicos_mes_long['tecnico'].str.contains('leonardo',case=False),
             enc_tecnicos_mes_long['tecnico'].str.contains('jaime',case=False)],
            ['ESNEIDER GARCIA BARAJAS','ALFONSO ARANGO TORO','SAIR BOLAÑO DURAN',
             'LEONARDO QUEZADA RAMIREZ','JAIME LEONARDO BARRERA BARAJAS'],
            default='')

enc_tecnicos_mes_long.loc[
    enc_tecnicos_mes_long['mes_crd']=='2024-08']

In [ ]:
#Ahora pego datos mikrowisp con encuestas de material
summary_tecnico_mes=mkwp_summary_month.merge(
    enc_tecnicos_mes_long,
    on=['mes_crd','tecnico'],how='right').fillna(0)

summary_tecnico_mes.loc[
    :,'% encuesta material'
]=summary_tecnico_mes[
    'Encuestas respondidas']/summary_tecnico_mes[
    'tickets_asignados']

summary_tecnico_mes.loc[
    :,'% encuesta satisfaccion'
]=summary_tecnico_mes[
    'num_enc_satisfaccion']/summary_tecnico_mes[
    'tickets_asignados']

summary_tecnico_mes.loc[
    :,'Satisfaccion del usuario'
]=summary_tecnico_mes[
    'Promotores']/summary_tecnico_mes[
    'num_enc_satisfaccion']

summary_tecnico_mes.loc[
    summary_tecnico_mes['mes_crd']=='2024-08'
]

## Gráficos

Numero de tickets por tecnico en el último mes

In [ ]:
meses_estudio=list(summary_tecnico_mes['mes_crd'].unique())[6:]

fig, ax=plt.subplots()

summary_tecnico_mes.loc[
    summary_tecnico_mes[
    'mes_crd']==meses_estudio[-2]
].plot.bar(
    'tecnico','tickets_asignados',
    ax=ax, color='lightseagreen', 
    grid=True, legend=False)

ax.set_xticklabels([x for x in summary_tecnico_mes.loc[
                    summary_tecnico_mes['mes_crd']==meses_estudio[-2],
                    'tecnico']], rotation=45,size=7)

plt.setp(ax,title='Tickets asignados por ténico '+meses_estudio[-2],
        xlabel='')
plt.tight_layout()
plt.show()

In [ ]:
#Mes actual
meses_estudio=list(summary_tecnico_mes['mes_crd'].unique())[6:]

fig, ax=plt.subplots()

summary_tecnico_mes.loc[
    summary_tecnico_mes[
    'mes_crd']==meses_estudio[-1]
].plot.bar(
    'tecnico','tickets_asignados',
    ax=ax, color='lightseagreen', 
    grid=True, legend=False)

ax.set_xticklabels([x for x in summary_tecnico_mes.loc[
                    summary_tecnico_mes['mes_crd']==meses_estudio[-1],
                    'tecnico']], rotation=45,size=7)

plt.setp(ax,title='Tickets asignados por ténico '+meses_estudio[-1],
        xlabel='')
plt.tight_layout()
plt.show()

Tiempo de resolucion por tecnico en el último mes

In [ ]:
meses_estudio=list(summary_tecnico_mes['mes_crd'].unique())[6:]

fig, ax=plt.subplots()

summary_tecnico_mes.loc[
    summary_tecnico_mes[
    'mes_crd']==meses_estudio[-2]
].plot.bar(
    'tecnico','temp_res',
    ax=ax, color='red', 
    grid=True, legend=False)

ax.set_xticklabels([x for x in summary_tecnico_mes.loc[
                    summary_tecnico_mes['mes_crd']==meses_estudio[-2],
                    'tecnico']], rotation=45,size=7)

plt.setp(ax,title='Tiempo de resolución por ténico '+meses_estudio[-2],
        xlabel='', ylabel='Días')

plt.tight_layout()
plt.show()

Mes actual

In [ ]:
meses_estudio=list(summary_tecnico_mes['mes_crd'].unique())[6:]

fig, ax=plt.subplots()

summary_tecnico_mes.loc[
    summary_tecnico_mes[
    'mes_crd']==meses_estudio[-1]
].plot.bar(
    'tecnico','temp_res',
    ax=ax, color='red', 
    grid=True, legend=False)

ax.set_xticklabels([x for x in summary_tecnico_mes.loc[
                    summary_tecnico_mes['mes_crd']==meses_estudio[-1],
                    'tecnico']], rotation=45,size=7)

plt.setp(ax,title='Tiempo de resolución por ténico '+meses_estudio[-1],
        xlabel='', ylabel='Días')

plt.tight_layout()
plt.show()

In [ ]:
meses_estudio=list(summary_tecnico_mes['mes_crd'].unique())[6:]

fig, ax=plt.subplots()

summary_tecnico_mes.loc[
    summary_tecnico_mes[
    'mes_crd']==meses_estudio[-2]
].plot.bar(
    'tecnico','% encuesta material',
    ax=ax, color='darkgreen', 
    grid=True, legend=False)

ax.set_xticklabels([x for x in summary_tecnico_mes.loc[
                    summary_tecnico_mes['mes_crd']==meses_estudio[-2],
                    'tecnico']], rotation=45,size=7)

plt.setp(ax,title='% encuestas material diligenciadas por ténico '+meses_estudio[-2],
        xlabel='', ylabel='')

import matplotlib.ticker as mtick
bars = ax.containers[0]
ax.bar_label(bars, labels = [f'{x.get_height():.0%}' for x in bars])
#ax.yaxis.set_major_formatter(mtick.PercentFormatter())
plt.tight_layout()
plt.show()

Mes actual

In [ ]:
meses_estudio=list(summary_tecnico_mes['mes_crd'].unique())[6:]

fig, ax=plt.subplots()

summary_tecnico_mes.loc[
    summary_tecnico_mes[
    'mes_crd']==meses_estudio[-1]
].plot.bar(
    'tecnico','% encuesta material',
    ax=ax, color='darkgreen', 
    grid=True, legend=False)

ax.set_xticklabels([x for x in summary_tecnico_mes.loc[
                    summary_tecnico_mes['mes_crd']==meses_estudio[-1],
                    'tecnico']], rotation=45,size=7)

plt.setp(ax,title='% encuestas material diligenciadas por ténico '+meses_estudio[-1],
        xlabel='', ylabel='')

import matplotlib.ticker as mtick
bars = ax.containers[0]
ax.bar_label(bars, labels = [f'{x.get_height():.0%}' for x in bars])
#ax.yaxis.set_major_formatter(mtick.PercentFormatter())
plt.tight_layout()
plt.show()

In [ ]:
meses_estudio=list(summary_tecnico_mes['mes_crd'].unique())[6:]

fig, ax=plt.subplots()

summary_tecnico_mes.loc[
    summary_tecnico_mes[
    'mes_crd']==meses_estudio[-2]
].plot.bar(
    'tecnico','num_enc_satisfaccion',
    ax=ax, color='blue', 
    grid=True, legend=False)

ax.set_xticklabels([x for x in summary_tecnico_mes.loc[
                    summary_tecnico_mes['mes_crd']==meses_estudio[-2],
                    'tecnico']], rotation=45,size=7)

plt.setp(ax,title='Num. encuestas satisfacción por ténico '+meses_estudio[-2],
        xlabel='', ylabel='')

import matplotlib.ticker as mtick
bars = ax.containers[0]
ax.bar_label(bars, labels = [f'{x.get_height()}' for x in bars])
#ax.yaxis.set_major_formatter(mtick.PercentFormatter())
plt.tight_layout()
plt.show()

In [ ]:
meses_estudio=list(summary_tecnico_mes['mes_crd'].unique())[6:]

fig, ax=plt.subplots()

summary_tecnico_mes.loc[
    summary_tecnico_mes[
    'mes_crd']==meses_estudio[-2]
].plot.bar(
    'tecnico','promedio_satisfaccion',
    ax=ax, color='blue', 
    grid=True, legend=False)

ax.set_xticklabels([x for x in summary_tecnico_mes.loc[
                    summary_tecnico_mes['mes_crd']==meses_estudio[-2],
                    'tecnico']], rotation=45,size=7)

plt.setp(ax,title='% encuestas material diligenciadas por ténico '+meses_estudio[-2],
        xlabel='', ylabel='')

import matplotlib.ticker as mtick
bars = ax.containers[0]
ax.bar_label(bars, labels = [f'{x.get_height():.3}' for x in bars])
#ax.yaxis.set_major_formatter(mtick.PercentFormatter())
plt.tight_layout()
plt.show()

Mes anterior

In [ ]:
#Satisfacción promedio por tecnico
summary_tecnico_mes.loc[
    summary_tecnico_mes[
    'mes_crd']==meses_estudio[-2],
    ['tecnico','num_enc_satisfaccion','promedio_satisfaccion']
]

Mes actual

In [ ]:
#Satisfacción promedio por tecnico
summary_tecnico_mes.loc[
    summary_tecnico_mes[
    'mes_crd']==meses_estudio[-1],
    ['tecnico','num_enc_satisfaccion','promedio_satisfaccion']
]

## Cuadro resumen por técnico/mes

In [ ]:
meses_estudio=list(summary_tecnico_mes['mes_crd'].unique())[6:]

tecnicos_instaladores=list(summary_tecnico_mes.loc[
    summary_tecnico_mes['mes_crd']=='2024-08',
    'tecnico'
].unique())[:-1]

###ESCALAS DE DESEMPEÑO

resumen_tecnicos_inst={}
for m in meses_estudio:
    
    filepath=os.path.join(
    resultados,
    'Indicadores_tecnicos_instaladores_'+m+'.xlsx')
    
    writer = pd.ExcelWriter(filepath, 
                        engine = 'xlsxwriter') 
    
    resumen_tecnicos_inst[m]={}
    
    for t in tecnicos_instaladores:
        resumen_tecnicos_inst[m][t]=pd.DataFrame()
        resumen_tecnicos_inst[m][t].loc[
            :,t]=summary_tecnico_mes.loc[
            (summary_tecnico_mes['mes_crd']==m) & 
            (summary_tecnico_mes['tecnico']==t)].iloc[0].loc[[
                'mes_crd','tickets_asignados',
                'Encuestas respondidas','% encuesta material',
                'num_enc_satisfaccion','% encuesta satisfaccion',
                'Satisfaccion del usuario','promedio_satisfaccion','temp_res']]


        ###OJO META DE LOS TÉCNICOS MENSUAL
        resumen_tecnicos_inst[m][t].loc[
                :,'Meta']=[m,None,None,
                           
                           #PORCENTAJE ENCUESTAS MATERIAL RESPONDIDAS
                           .7,None,None,
                           #PROMOTORES
                           .7,None,
                           #TIEMPO RESOLUCIÓN
                           pd.to_timedelta(3.5, unit='d')]


        ##PORCENTAJE DE CUMPLIMIENTO
        resumen_tecnicos_inst[m][t].loc[:,
        '% Cumplimiento']=resumen_tecnicos_inst[m][t].loc[
        (resumen_tecnicos_inst[m][t].index=='% encuesta material') |
        (resumen_tecnicos_inst[m][t].index=='Satisfaccion del usuario')][m][t]/resumen_tecnicos_inst[m][t].loc[
        (resumen_tecnicos_inst[m][t].index=='% encuesta material') |
        (resumen_tecnicos_inst[m][t].index=='Satisfaccion del usuario')]['Meta']

        resumen_tecnicos_inst[m][t].loc[
                resumen_tecnicos_inst[m][t].index=='temp_res',
        '% Cumplimiento']=(1-(resumen_tecnicos_inst[m][t].loc[
                resumen_tecnicos_inst[m][t].index=='temp_res'][m][t]-
                              resumen_tecnicos_inst[m][t].loc[
                resumen_tecnicos_inst[m][t].index=='temp_res']['Meta'])/resumen_tecnicos_inst[m][t].loc[
                resumen_tecnicos_inst[m][t].index=='temp_res']['Meta'])

        ###CALIFICACION DEL DESEMPEÑO
        resumen_tecnicos_inst[m][t].loc[:,
        'Desempeño']=np.select([resumen_tecnicos_inst[m][t]['% Cumplimiento']>=1,
                                (resumen_tecnicos_inst[m][t]['% Cumplimiento']>=.8) &
                                (resumen_tecnicos_inst[m][t]['% Cumplimiento']<1), 
                                (resumen_tecnicos_inst[m][t]['% Cumplimiento']>=.5) &
                                (resumen_tecnicos_inst[m][t]['% Cumplimiento']<.8), 
                                (resumen_tecnicos_inst[m][t]['% Cumplimiento']>=0) &
                                (resumen_tecnicos_inst[m][t]['% Cumplimiento']<.5)],
                               ['Excelente','Destacado','Oportunidad','Crítico'], default='')
        
        resumen_tecnicos_inst[m][t].loc[
        'Promedio','% Cumplimiento']=resumen_tecnicos_inst[m][t][
        '% Cumplimiento'].mean()

        resumen_tecnicos_inst[m][t].loc[
        'Promedio','Desempeño']=np.select([resumen_tecnicos_inst[t].loc[
                                           'Promedio','% Cumplimiento']>=1,
                                (resumen_tecnicos_inst[t].loc[
                                           'Promedio','% Cumplimiento']>=.8) &
                                (resumen_tecnicos_inst[t].loc[
                                           'Promedio','% Cumplimiento']<1), 
                                (resumen_tecnicos_inst[t].loc[
                                           'Promedio','% Cumplimiento']>=.5) &
                                (resumen_tecnicos_inst[t].loc[
                                           'Promedio','% Cumplimiento']<.8), 
                                (resumen_tecnicos_inst[t].loc[
                                           'Promedio','% Cumplimiento']>=0) &
                                (resumen_tecnicos_inst[t].loc[
                                           'Promedio','% Cumplimiento']<.5)],
                               ['Excelente','Destacado','Oportunidad','Crítico'], default='')
        
        #resumen_tecnicos_inst[t].fillna(0, inplace=True)

        resumen_tecnicos_inst[t].index=['Mes','Tickets asignados','Encuestas material', '% encuesta material respondidas',
                                                       'Num. encuestas satisfaccion','% Enc. sat. respondidas', 'Satisfaccion del usuario',
                                                       'Promedio satisfacción','Tiempo resolución','Promedio']
        
        resumen_tecnicos_inst[t].to_excel(
                writer, sheet_name=t)

    writer.close()